In [1]:
import torch
import torch.nn as nn
import torchtext
from torchtext import data
from torchtext.vocab import Vectors
from torchtext.data import Iterator, BucketIterator
import spacy
import numpy as np


In [2]:
import jsonlines

In [6]:
with open('data/snli_1.0/snli_1.0_dev.jsonl', 'r+', encoding='utf-8') as f:
    for idx, item in enumerate(jsonlines.Reader(f)):
        print(item.keys())
        print(item.values())
        print(item)
        print((type(item)))
        if idx > 0:
            break

dict_keys(['annotator_labels', 'captionID', 'gold_label', 'pairID', 'sentence1', 'sentence1_binary_parse', 'sentence1_parse', 'sentence2', 'sentence2_binary_parse', 'sentence2_parse'])
dict_values([['neutral', 'entailment', 'neutral', 'neutral', 'neutral'], '4705552913.jpg#2', 'neutral', '4705552913.jpg#2r1n', 'Two women are embracing while holding to go packages.', '( ( Two women ) ( ( are ( embracing ( while ( holding ( to ( go packages ) ) ) ) ) ) . ) )', '(ROOT (S (NP (CD Two) (NNS women)) (VP (VBP are) (VP (VBG embracing) (SBAR (IN while) (S (NP (VBG holding)) (VP (TO to) (VP (VB go) (NP (NNS packages)))))))) (. .)))', 'The sisters are hugging goodbye while holding to go packages after just eating lunch.', '( ( The sisters ) ( ( are ( ( hugging goodbye ) ( while ( holding ( to ( ( go packages ) ( after ( just ( eating lunch ) ) ) ) ) ) ) ) ) . ) )', '(ROOT (S (NP (DT The) (NNS sisters)) (VP (VBP are) (VP (VBG hugging) (NP (UH goodbye)) (PP (IN while) (S (VP (VBG holding) (S (VP (T

In [2]:
spcay_load='en_core_web_sm'
nlp = spacy.load(spcay_load)

In [3]:
tokenizer_result = nlp.tokenizer('nlp is interesting')

In [4]:
type(tokenizer_result)

spacy.tokens.doc.Doc

In [5]:
doc_result = nlp('nlp is interesting')

In [6]:
type(doc_result)

spacy.tokens.doc.Doc

In [7]:
tokenizer_result == doc_result

False

In [8]:
tokenize = lambda x: [tok.text for tok in nlp.tokenizer(x) if not tok.is_punct | tok.is_space]

In [9]:
tokenize('nlp is interesting, isn\'t it?')

['nlp', 'is', 'interesting', 'is', "n't", 'it']

In [10]:
tokenize('nlp is interesting, hahaha!')

['nlp', 'is', 'interesting', 'hahaha']

In [27]:
vectors = Vectors(name='glove.840B.300d.txt')
vectors.unk_init = nn.init.xavier_uniform_

In [28]:
dir(vectors)[-10:]

['__str__',
 '__subclasshook__',
 '__weakref__',
 'cache',
 'dim',
 'get_vecs_by_tokens',
 'itos',
 'stoi',
 'unk_init',
 'vectors']

In [16]:
type(vectors.stoi)

dict

In [17]:
type(vectors.stoi)

dict

In [29]:
vectors.vectors.shape

torch.Size([2196017, 300])

In [22]:
for idx, (key, value) in enumerate(vectors.stoi.items()):
    print('{key}  {value}'.format(key=key, value=value))
    if idx > 5:
        break

,  0
.  1
the  2
and  3
to  4
of  5
a  6


In [26]:
a = torch.rand((2, 3), requires_grad=True)

In [27]:
a.data

tensor([[0.8525, 0.7719, 0.1609],
        [0.3858, 0.2812, 0.3295]])

In [28]:
a

tensor([[0.8525, 0.7719, 0.1609],
        [0.3858, 0.2812, 0.3295]], requires_grad=True)

In [29]:
a.detach()

tensor([[0.8525, 0.7719, 0.1609],
        [0.3858, 0.2812, 0.3295]])

In [30]:
a.detach() == a.data

tensor([[True, True, True],
        [True, True, True]])

In [36]:
a.unsqueeze(1).shape

torch.Size([2, 1, 3])

In [31]:
drop_out_net = nn.Dropout()

In [34]:
drop_out_net.training

True

In [35]:
dir(drop_out_net)

['__call__',
 '__class__',
 '__constants__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_name',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_modules',
 '_named_members',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_slow_forward',
 '_state_dict_hooks',
 '_version',
 'add_module',
 'apply',
 'bfloat16',
 'buffers',
 'children',
 'cpu',
 'cuda',
 'double',
 'dump_patches',
 'eval',
 'extra_repr',
 'float',
 'forward',
 'half',
 'inplace',
 'load_

In [15]:
input_tensor = torch.rand((2, 3, 3))  # [b, length, length]
input_tensor

tensor([[[0.3232, 0.3057, 0.7403],
         [0.6914, 0.0307, 0.1169],
         [0.4783, 0.4467, 0.3348]],

        [[0.2118, 0.6903, 0.5553],
         [0.0477, 0.1964, 0.0776],
         [0.4860, 0.3096, 0.0040]]])

In [22]:
mask = torch.tensor([[1, 0, 0], [1, 1, 1]])
mask

tensor([[1, 0, 0],
        [1, 1, 1]])

In [17]:
 tensor_shape = input_tensor.shape
 reshaped_tensor = input_tensor.view(-1, tensor_shape[-1])
 reshaped_tensor.shape

torch.Size([6, 3])

In [18]:
reshaped_tensor.data_ptr() == input_tensor.data_ptr()

True

In [23]:
 while mask.dim() < input_tensor.dim():
    mask = mask.unsqueeze(1)
mask.shape

torch.Size([2, 1, 3])

In [24]:
mask = mask.expand_as(input_tensor).contiguous().float()
mask.shape

torch.Size([2, 3, 3])

In [25]:
mask

tensor([[[1., 0., 0.],
         [1., 0., 0.],
         [1., 0., 0.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]])

In [26]:
reshaped_mask = mask.view(-1, mask.size()[-1])
reshaped_mask

tensor([[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])

In [35]:
weight = torch.FloatTensor([[1, 2.3, 3], [4, 5.1, 6.3]])
embedding = nn.Embedding.from_pretrained(weight, freeze=False, padding_idx=0)

In [36]:
embedding.weight

Parameter containing:
tensor([[1.0000, 2.3000, 3.0000],
        [4.0000, 5.1000, 6.3000]], requires_grad=True)

In [37]:
embedding.weight[0, :] = 0

In [38]:
embedding.weight

Parameter containing:
tensor([[0.0000, 0.0000, 0.0000],
        [4.0000, 5.1000, 6.3000]], grad_fn=<CopySlices>)

In [34]:
embedding.weight.requires_grad

False

In [39]:
embedding1 = nn.Embedding(2, 3, padding_idx=0)

In [41]:
embedding1.weight.requires_grad

True

In [43]:
mask.shape

torch.Size([2, 3, 3])

In [45]:
premise_mask = torch.tensor([[1, 0, 0], [1, 1, 1]])
torch.sum(premise_mask, dim=1, keepdim=True)

tensor([[1],
        [3]])

In [47]:
v_ai = torch.rand((2, 3, 4))
v_ai

tensor([[[0.3540, 0.3405, 0.6841, 0.4523],
         [0.1541, 0.1414, 0.9908, 0.4920],
         [0.4930, 0.9657, 0.5728, 0.1380]],

        [[0.2912, 0.9040, 0.5434, 0.7209],
         [0.9695, 0.4247, 0.7810, 0.7691],
         [0.6388, 0.8017, 0.6768, 0.6810]]])

In [48]:
v_ai.max(dim=1)

torch.return_types.max(
values=tensor([[0.4930, 0.9657, 0.9908, 0.4920],
        [0.9695, 0.9040, 0.7810, 0.7691]]),
indices=tensor([[2, 2, 1, 1],
        [1, 0, 1, 1]]))

In [49]:
v_ai.shape

torch.Size([2, 3, 4])

In [51]:
v_ai.size()[0]

2

In [55]:
dropout = nn.Dropout(0.5)
data = torch.tensor([[1.0, 2.0, 3.0], [1.0, 2.0, 3.0]])
dropout(data)

tensor([[2., 0., 0.],
        [2., 4., 6.]])